# Analizador sintáctico

In [1]:
import sys
import re

class Token:
  def __init__(self, lexema, tipo, fila, col):
    self.lexema = lexema
    self.tipo = tipo
    self.fila = fila
    self.col = col


In [2]:
class Regla:    
    def __init__(self,no_terminal,derivacion):
        self.no_terminal = no_terminal
        self.derivacion = derivacion
        
    def _primeros(self,alfa,gramatica):
        prim = set()
        if alfa == ['Ñ']:
            prim.add('Ñ')
            return prim
        else:
            a1 = alfa[0]
            if a1.isupper():
                prim.add(a1)
                return prim
            else:
                reglas_de_a1 = gramatica[a1]
                primeros_a1 = set()
                for regla in reglas_de_a1:
                    primeros_a1 = primeros_a1 | self._primeros(regla.derivacion,gramatica)                
                copia_primeros_a1 = primeros_a1.copy()
                copia_primeros_a1.discard('Ñ')                
                prim = prim | copia_primeros_a1                
                if 'Ñ' in primeros_a1:                                   
                    if len(alfa) == 1:
                        prim.add('Ñ')                        
                        return prim
                    else:                         
                        alfa2 = alfa.copy()
                        alfa2.pop(0)                        
                        prim = prim | self._primeros(alfa2,gramatica)                         
        return prim

    def _siguientes(self,A,gramatica):
        sig = set()
        if A == list(gramatica.keys())[0]:
            sig.add('ÑÑ')
        for no_terminal in gramatica.keys():
            reglas = gramatica[no_terminal]
            for regla in reglas: 
                der = regla.derivacion
                if A in der:                    
                    while True:
                        len1 = len(sig)
                        if der[-1] == A:                            
                            sig = sig | self._siguientes(no_terminal,gramatica)                            
                        else:
                            lista = []
                            lista.append(der[der.index(A)+1])
                            primeros_de_beta = self._primeros(lista,gramatica)                            
                            primeros_de_beta2 = primeros_de_beta.copy()
                            primeros_de_beta2.discard('Ñ')
                            sig = sig | primeros_de_beta2                            
                            if 'Ñ' in primeros_de_beta:
                                sig = sig | self._siguientes(no_terminal,gramatica)                                
                        len2 = len(sig)
                        if len1 == len2:
                            return sig
        return sig

    def conjunto_prediccion(self,gramatica):
        conj_pred = set()
        alfa = self.derivacion
        no_terminal = self.no_terminal
        prim_alfa = self._primeros(alfa,gramatica)
        if 'Ñ' in prim_alfa:            
            prim_alfa.discard('Ñ')
            conj_pred = prim_alfa | self._siguientes(no_terminal,gramatica)
        else:
            conj_pred = self._primeros(alfa,gramatica)
        return conj_pred
    

In [3]:
gramatica = {'program': [Regla('program',['component','component_ls'])],
             'component_ls': [Regla('component_ls',['component','component_ls']),Regla('component_ls',['Ñ'])],
             'component': [Regla('component',['spec_component']),Regla('component',['combined_component']),Regla('component',['separate_body']),Regla('component',['error'])],
             'spec_component': [Regla('spec_component',['comp_label','spec_stmt_ls','spec_body'])],
             'combined_component': [Regla('combined_component',['combined_specpart','body_stmt_ls','end_id'])],
             'combined_specpart': [Regla('combined_specpart',['comp_label','parameters'])],
             'comp_label': [Regla('comp_label',['comp_kwd','TK_ID'])],             
             'comp_kwd': [Regla('comp_kwd',['TK_GLOBAL']),Regla('comp_kwd',['TK_RESOURCE'])],
             'spec_body': [Regla('spec_body',['end_id']),Regla('spec_body',['TK_BODY','TK_ID','maybe_params','TK_SEPARATE']),Regla('spec_body',['TK_BODY','TK_ID','maybe_params','body_stmt_ls','end_id'])],
             'maybe_params': [Regla('maybe_params',['Ñ']),Regla('maybe_params',['parameters'])],
             'separate_body': [Regla('separate_body',['TK_BODY','TK_ID','body_stmt_ls','end_id'])],
             'spec_stmt_ls': [Regla('spec_stmt_ls',['spec_stmt','spec_stmt_ls2'])], 
             'spec_stmt_ls2': [Regla('spec_stmt_ls2',['Ñ']),Regla('spec_stmt_ls2',['TK_SEPARATOR','spec_stmt','spec_stmt_ls2'])],
             'spec_stmt': [Regla('spec_stmt',['common_stmt']),Regla('spec_stmt',['extend_clause']),Regla('spec_stmt',['body_only'])],
             'body_stmt_ls': [Regla('body_stmt_ls',['body_stmt','body_stmt_ls2'])],
             'body_stmt_ls2': [Regla('body_stmt_ls2',['TK_SEPARATOR','body_stmt','body_stmt_ls2']),Regla('spec_stmt_ls2',['Ñ'])],
             'body_stmt': [Regla('body_stmt',['common_stmt']),Regla('body_stmt',['expr']),Regla('body_stmt',['body_only']),Regla('body_stmt',['extend_clause'])],
             'body_only': [Regla('body_only',['stmt']),Regla('body_only',['proc']),Regla('body_only',['process']),Regla('body_only',['procedure']),Regla('body_only',['initial_block']),Regla('body_only',['final_block'])],
             'common_stmt': [Regla('common_stmt',['Ñ']),Regla('common_stmt',['decl']),Regla('common_stmt',['import_clause'])],
             'import_clause': [Regla('import_clause',['TK_IMPORT','import_list'])],
             'extend_clause': [Regla('extend_clause',['TK_EXTEND','import_list'])],
             'import_list': [Regla('import_list',['import_name','import_list2'])],
             'import_list2': [Regla('import_list2',['TK_COMMA','import_name','import_list2']),Regla('import_list2',['Ñ'])],
             'import_name': [Regla('import_name',['TK_ID'])],
             'end_id': [Regla('end_id',['TK_END','id_opt'])],
             'id_opt': [Regla('id_opt',['Ñ']),Regla('id_opt',['TK_ID'])],
             
             'decl': [Regla('decl',['error','TK_SEPARATOR']),Regla('decl',['type_decl']),Regla('decl',['obj_decl']),Regla('decl',['optype_decl']),Regla('decl',['op_decl'])],
             'type_decl': [Regla('type_decl',['TK_TYPE','TK_ID','TK_EQ','type','type_restriction'])],
             'type_restriction': [Regla('type_restriction',['TK_LBRACE','TK_ID','TK_RBRACE']),Regla('type_restriction',['Ñ'])],
             'obj_decl': [Regla('obj_decl',['var_or_const','var_def_lp'])],
             'var_or_const': [Regla('var_or_const',['TK_VAR']),Regla('var_or_const',['TK_CONST'])],
             'var_def_lp': [Regla('var_def_lp',['var_def','var_def_lp2'])],
             'var_def_lp2': [Regla('var_def_lp2',['Ñ']),Regla('var_def_lp2',['TK_COMMA','var_def','var_def_lp2'])],
             'var_def': [Regla('var_def',['id_subs_lp','var_att'])],
             'var_att': [Regla('var_att',['TK_COLON','type','var_att2']),Regla('var_att',['TK_SEPARATOR']),Regla('var_att',['TK_ASSIGN','expr'])],
             'var_att2': [Regla('var_att2',['TK_ASSIGN','expr']),Regla('var_att2',['Ñ'])],
             'id_subs_lp': [Regla('id_subs_lp',['id_subs','id_subs_lp2'])],
             'id_subs_lp2': [Regla('id_subs_lp2',['TK_COMMA','id_subs','id_subs_lp2']),Regla('id_subs_lp2',['Ñ'])],
             'id_subs': [Regla('id_subs',['TK_ID','id_subs2'])],
             'id_subs2': [Regla('id_subs2',['subscripts']),Regla('id_subs2',['Ñ'])],
             'subscripts': [Regla('subscripts',['bracketed_list','subscripts2'])],
             'subscripts2': [Regla('subscripts2',['subscripts']),Regla('subscripts2',['Ñ'])],
             'bracketed_list': [Regla('bracketed_list',['TK_LBRACKET','bound_lp','TK_RBRACKET'])],
             'bound_lp': [Regla('bound_lp',['bounds','bound_lp2'])],
             'bound_lp2': [Regla('bound_lp2',['TK_COMMA','bounds','bound_lp2']),Regla('bound_lp2',['Ñ'])],
             'bounds': [Regla('bounds',['bound','bounds2'])],
             'bounds2': [Regla('bounds2',['TK_COLON','bound']),Regla('bounds2',['Ñ'])],
             'bound': [Regla('bound',['expr']),Regla('bound',['TK_ASTER'])],

             'expr': [Regla('expr',['TK_ID','expr2']),Regla('expr',['num','expr2'])],
             'num': [Regla('num',['TK_INT']),Regla('num',['TK_REAL'])],
             'expr2': [Regla('expr2',['operator','expr']),Regla('expr2',['Ñ'])],
             'operator': [Regla('operator',['TK_PLUS']),Regla('operator',['TK_MINUS'])],
             #'binary_expr': [Regla('binary_expr',['operator','expr'])],
             #'binary_expr2': [Regla('binary_expr2',['TK_PLUS','expr']),Regla('binary_expr2',['TK_MINUS','expr'])],
             
             'optype_decl': [Regla('optype_decl',['TK_ID'])],
             'op_decl': [Regla('op_decl',['TK_ID'])],
             'error': [Regla('error',['TK_ID'])],
             'type': [Regla('type',['TK_ID'])],
             'comp_params': [Regla('comp_params',['TK_ID'])],             
             'parameters': [Regla('parameters',['TK_ID'])],
             'stmt': [Regla('stmt',['TK_ID'])],
             'proc': [Regla('proc',['TK_ID'])],
             'process': [Regla('process',['TK_ID'])],
             'procedure': [Regla('procedure',['TK_ID'])],
             'initial_block': [Regla('initial_block',['TK_ID'])],
             'final_block': [Regla('final_block',['TK_ID'])]
             
             
}
gramatica

{'program': [<__main__.Regla at 0x1be49238668>],
 'component_ls': [<__main__.Regla at 0x1be492386a0>,
 'component': [<__main__.Regla at 0x1be49238710>,
 'spec_component': [<__main__.Regla at 0x1be492387f0>],
 'combined_component': [<__main__.Regla at 0x1be49238828>],
 'combined_specpart': [<__main__.Regla at 0x1be49238860>],
 'comp_label': [<__main__.Regla at 0x1be49238898>],
 'comp_kwd': [<__main__.Regla at 0x1be492388d0>,
 'spec_body': [<__main__.Regla at 0x1be49238940>,
 'maybe_params': [<__main__.Regla at 0x1be492389e8>,
 'separate_body': [<__main__.Regla at 0x1be49238a58>],
 'spec_stmt_ls': [<__main__.Regla at 0x1be49238a90>],
 'spec_stmt_ls2': [<__main__.Regla at 0x1be49238ac8>,
 'spec_stmt': [<__main__.Regla at 0x1be49238b38>,
 'body_stmt_ls': [<__main__.Regla at 0x1be49238be0>],
 'body_stmt_ls2': [<__main__.Regla at 0x1be49238c18>,
 'body_stmt': [<__main__.Regla at 0x1be49238c88>,
 'body_only': [<__main__.Regla at 0x1be49238d68>,
 'common_stmt': [<__main__.Regla at 0x1be49238eb

In [4]:
tipo_tk_operador = dict([('|', 'TK_OR'), ('&', 'TK_AND'), ('?', 'TK_QMARK'), ('@', 'TK_ADDR'), ('=', 'TK_EQ'), ('<', 'TK_LT'), ('~', 'TK_NOT'), ('>', 'TK_GT'), ('-', 'TK_MINUS'), ('+', 'TK_PLUS'), ('*', 'TK_ASTER'), ('/', 'TK_DIV'), ('^', 'TK_HAT'),
                  ('%', 'TK_REMDR'), ('.', 'TK_PERIOD'), (',', 'TK_COMMA'), (';', 'TK_SEPARATOR'), (':', 'TK_COLON'), ('{', 'TK_LBRACE'), ('}', 'TK_RBRACE'), ('(', 'TK_LPAREN')
                  , (')', 'TK_RPAREN'), ('[', 'TK_LBRACKET'), (']', 'TK_RBRACKET'),('...', 'TK_THREE_PERIODS'), (':=:', 'TK_SWAP'), ('--', 'TK_DECR'), ('++', 'TK_INCR'), ('**', 'TK_EXPON'), (':=', 'TK_ASSIGN'), ('->', 'TK_ARROW'), ('[]', 'TK_SQUARE'), 
                ('//', 'TK_PARALLEL'), ('<=', 'TK_LE'),('>=', 'TK_GE'),('!=', 'TK_NE'),('Fin de archivo', 'ÑÑ')])
tipo_tk_operador_inv = dict((v,k) for k,v in tipo_tk_operador.items())

In [5]:
def lexer():
    fila = 1
    col = 1

    tokens = []

    with open(r'C:\Users\corre\Desktop\my_file.txt','r') as file:
        codigoFuente = file.read()

    #CaseSensitive
    pSaltoDeLinea = re.compile(r'\n', re.M)
    pComentarios = re.compile(r'#(.*)?')
    pCadena = re.compile(r'("[^"]*")|(\'[^\']*\')|(“[^”]*”)')
    pReservadas = re.compile(r'\btrue\b | \bfalse\b | \baf\b | \breal\b | \bglobal\b | \bnext\b | \bstring\b | \bvar\b | \bint\b | \bbody\b | \bgetarg\b | \bif\b | \bmod\b | \bwrite\b | \bwrites\b | \bread\b | \bstop\b | \bfi\b | \bend\b | \bresource\b | \bprocedure\b | \breturns\b | \bor\b | \band\b | \bfa\b | \bto\b | \bimport\b | \bcap\b | \bcreate\b | \bsend\b | \binitial\b | \bfinal\b | \bop\b | \bnew\b | \bexternal\b  | \bproc\b | \belse\b | \bextend\b | \bdestroy\b | \bon\b | \bnull\b | \bnoop\b | \bin\b | \bres\b | \bref\b | \bby\b | \bselect\b | \bdo\b | \bod\b | \bni\b | \bprocess\b | \breceive\b | \bcall\b | \bni\b | \breply\b | \bexit\b | \bco\b | \boc\b | \bwhen\b | \bbinding\b | \boptype\b | \bst\b | \bfile\b | \bbool\b | \bfrom\b | \bchar\b | \bconst\b | \btype\b | \brec\b | \bprivate\b | \babort\b | \bselect\b | \bthen\b | \bcoenter\b | \bcoenter\b', re.X)
    pIdentificador = re.compile(r'\b[_a-zA-Z][\w_]{0,31}\b')
    pEntero = re.compile(r'[-+]?\d+((E-|E\+|e-|e\+)\d+)?')
    pReal = re.compile(r'\d+\.\d+((E-|E\+|e-|e\+)\d+)?')
    pOperadores3 = re.compile(r'(\:\=\:)|(\.\.\.)')
    pOperadores2 = re.compile(r'(\*\*)|(\-\-)|(\+\+)|(\/\/)|(\<\=)|(\>\=)|(\<\>)|(\:\=)|(\-\>)|(\[\])|(\!\=)')
    pOperadores = re.compile(r'[\|\&\?\@\~\+\-\*\/\^\=\<\>\%\.\,\;\:\{\}\[\]\(\)]')
    pEspacio = re.compile(r'\s')   


    while (codigoFuente != ''):

      if(pSaltoDeLinea.match(codigoFuente)):
          lex = pSaltoDeLinea.match(codigoFuente).group()
          codigoFuente = codigoFuente[len(lex):]
          fila += 1
          col = 1

      elif (pComentarios.match(codigoFuente)):
        lex = pComentarios.match(codigoFuente).group()
        codigoFuente = codigoFuente[len(lex):]

      elif (pCadena.match(codigoFuente)):
        lex = pCadena.match(codigoFuente).group()
        codigoFuente = codigoFuente[len(lex):]
        token =  Token(lex, 'TK_STRING', fila, col)
        tokens.append(token)
        col = col + len(lex)
        print("<" + token.tipo + "," + token.lexema + "," + str(token.fila) + "," + str(token.col) + ">")

      elif (pReservadas.match(codigoFuente)):
        lex = pReservadas.match(codigoFuente).group()
        codigoFuente = codigoFuente[len(lex):]
        token =  Token(lex, ('TK_'+lex.upper()), fila, col)
        tokens.append(token)
        col = col + len(lex)
        print("<" + token.tipo + "," + str(token.fila) + "," + str(token.col) + ">")

      elif (pIdentificador.match(codigoFuente)):
          lex = pIdentificador.match(codigoFuente).group()
          codigoFuente = codigoFuente[len(lex):]
          token = Token(lex, 'TK_ID', fila, col)
          tokens.append(token)
          col = col + len(lex)
          print("<" + token.tipo + "," + token.lexema + "," + str(token.fila) + "," + str(token.col) + ">")
          if (pOperadores.match(codigoFuente)):
            lex = pOperadores.match(codigoFuente).group()
            codigoFuente = codigoFuente[len(lex):]
            token = Token(lex, tipo_tk_operador[lex], fila, col)
            tokens.append(token)
            col = col + len(lex)
            print("<" + token.tipo + "," + str(token.fila) + "," + str(token.col) + ">")

      elif (pReal.match(codigoFuente)):
        lex = pReal.match(codigoFuente).group()
        codigoFuente = codigoFuente[len(lex):]
        token = Token(lex, 'TK_REAL', fila, col)
        tokens.append(token)
        col = col + len(lex)
        print("<" + token.tipo + "," + token.lexema + "," + str(token.fila) + "," + str(token.col) + ">")
        if (pOperadores.match(codigoFuente)):
          lex = pOperadores.match(codigoFuente).group()
          codigoFuente = codigoFuente[len(lex):]
          token = Token(lex, tipo_tk_operador[lex], fila, col)
          tokens.append(token)
          col = col + len(lex)
          print("<" + token.tipo + "," + str(token.fila) + "," + str(token.col) + ">")

      elif (pEntero.match(codigoFuente)):
        lex = pEntero.match(codigoFuente).group()
        codigoFuente = codigoFuente[len(lex):]
        token = Token(lex, 'TK_INT', fila, col)
        tokens.append(token)
        col = col + len(lex)
        print("<" + token.tipo + "," + token.lexema + "," + str(token.fila) + "," + str(token.col) + ">")
        if (pOperadores.match(codigoFuente)):
          lex = pOperadores.match(codigoFuente).group()
          codigoFuente = codigoFuente[len(lex):]
          token = Token(lex, tipo_tk_operador[lex], fila, col)
          tokens.append(token)
          col = col + len(lex)
          print("<" + token.tipo + "," + str(token.fila) + "," + str(token.col) + ">")

      elif (pOperadores3.match(codigoFuente)):
        lex = pOperadores3.match(codigoFuente).group()
        codigoFuente = codigoFuente[len(lex):]
        token = Token(lex, tipo_tk_operador[lex], fila, col)
        tokens.append(token)
        col = col + len(lex)
        print("<" + token.tipo + "," + str(token.fila) + "," + str(token.col) + ">")

      elif (pOperadores2.match(codigoFuente)):
        lex = pOperadores2.match(codigoFuente).group()
        codigoFuente = codigoFuente[len(lex):]
        token = Token(lex, tipo_tk_operador[lex], fila, col)
        tokens.append(token)
        col = col + len(lex)
        print("<" + token.tipo + "," + str(token.fila) + "," + str(token.col) + ">")

      elif (pOperadores.match(codigoFuente)):
        lex = pOperadores.match(codigoFuente).group()
        codigoFuente = codigoFuente[len(lex):]
        token =  Token(lex, tipo_tk_operador[lex], fila, col)
        tokens.append(token)
        col = col + len(lex)
        print("<" + token.tipo + "," + str(token.fila) + "," + str(token.col) + ">")

      elif (pEspacio.match(codigoFuente)):
        lex = pEspacio.match(codigoFuente).group()
        codigoFuente = codigoFuente[len(lex):]
        col = col + 1

      else:
          print(">>> Error lexico(linea:" + str(fila) + "," + "posicion:" + str(col) + ")")
          break

    token = Token('ÑÑ', 'ÑÑ', -1, -1)
    tokens.append(token)    
    return tokens

In [6]:
#new_regla = Regla('expr',['binary_expr'])
#conj_pred = new_regla.conjunto_prediccion(gramatica)
#conj_pred
   

In [7]:
def regla_a_aplicar(gramatica,no_terminal,token):    
    reglas = gramatica[no_terminal]    
    for regla in reglas: 
        conj_pred_regla = regla.conjunto_prediccion(gramatica)
        if token.tipo in conj_pred_regla:
            return regla
    return None

def sintaxError1(no_terminal,gramatica):
    simbolosEsperados = []
    conj_pred_regla = set()
    reglas = gramatica[no_terminal]    
    for regla in reglas: 
        conj_pred_regla = conj_pred_regla | regla.conjunto_prediccion(gramatica)
    for no_trmnl in conj_pred_regla:
        if no_trmnl in tipo_tk_operador_inv:
            simbolosEsperados.append(tipo_tk_operador_inv[no_trmnl])
        else:
            simbolosEsperados.append(no_trmnl)        
    return simbolosEsperados
        
                    

In [8]:
def derivar2(regla):
    global tokens
    global token_index
    print("regla1:")
    print(regla.derivacion)
    i = 0
    while i < len(regla.derivacion):  
        token = tokens[token_index]
        #print("i1:")
        #print(i)
        #print("regla2:")
        #print(regla.derivacion)
        simbolo = regla.derivacion[i]
        #print("simbolo")
        #print(simbolo)
        if simbolo.isupper():            
            #print("token1")
            #print(token.tipo)
            #print("tokens_index")
            #print(token_index)
            if simbolo == 'Ñ':
                i = i + 1
            else:
                if simbolo == token.tipo:
                    i = i + 1
                    token_index = token_index + 1                
                    #print("token2")
                    #print(token.tipo)
                    #print("i")
                    #print(i)
                else:
                    if simbolo == 'TK_RESOURCE':
                        print('Error sintactico: falta funcion_principal')
                    else:
                        print("<" + str(token.fila) +"," + str(token.col) + "> Error sintactico: se encontro: " + token.lexema +" ; se esperaba: " + simbolo)
                    return True                
        else:
            no_terminal = simbolo
            #print("tokens_index4:")
            #print(token_index)
            #print("noterminal")
            #print(no_terminal)
            #print("token4")
            #print(token.tipo)
            siguiente_regla = regla_a_aplicar(gramatica,no_terminal,token)
            if siguiente_regla == None:                
                simbolos_esperados = sintaxError1(no_terminal,gramatica)                    
                #print("error sintactico2")
                print("<" + str(token.fila) +"," + str(token.col) + "> Error sintactico: se encontro: " + token.lexema +" ; se esperaba: " + str(simbolos_esperados))
                return True                
            else:
                #print("regla a derivar:")
                #print(regla.derivacion)
                error = derivar2(siguiente_regla)
                i = i + 1
                #print("i3")
                #print(i)
                #print("regla4")
                #print(siguiente_regla.derivacion)
                if error:
                    return True                
    return False

In [10]:
tokens = lexer()
token_index = 0
simbolo_inicial = list(gramatica.keys())[0]
token = tokens[token_index]
siguiente_regla = regla_a_aplicar(gramatica,simbolo_inicial,token)
print("regla")
print(siguiente_regla.derivacion)
if siguiente_regla == None:
    #print("error sintactico2")
    simbolos_esperados = sintaxError1(no_terminal,gramatica)
    print("<" + str(token.fila) +"," + str(token.col) + "> Error sintactico: se encontro: " + token.lexema +" ; se esperaba: " + str(simbolos_esperados))    
else:
    error = derivar2(siguiente_regla)
    if not error:
        if tokens[token_index].tipo != 'ÑÑ':    
            print("error sintactico, se esperaba fin de archivo")
        else:
            print("El analisis sintactico ha finalizado exitosamente.") 


<TK_GLOBAL,3,1>
<TK_ID,sizes,3,8>
<TK_VAR,4,1>
<TK_ID,N,4,5>
<TK_ASSIGN,4,7>
<TK_INT,10,4,10>
<TK_SEPARATOR,4,12>
<TK_VAR,5,1>
<TK_ID,PR,5,5>
<TK_ASSIGN,5,8>
<TK_INT,2,5,11>
<TK_SEPARATOR,5,12>
<TK_END,6,1>
<TK_RESOURCE,8,1>
<TK_ID,a,8,10>
<TK_CONST,9,5>
<TK_ID,N,9,11>
<TK_COLON,9,13>
<TK_INT,12,9,15>
<TK_SEPARATOR,9,17>
<TK_END,10,1>
<TK_RESOURCE,12,1>
<TK_ID,b,12,10>
<TK_EXTEND,13,5>
<TK_ID,a,13,12>
<TK_SEPARATOR,13,13>
<TK_CONST,14,5>
<TK_ID,M,14,11>
<TK_ASSIGN,14,13>
<TK_INT,33,14,16>
<TK_PLUS,14,18>
<TK_INT,7,14,19>
<TK_PLUS,14,20>
<TK_ID,N,14,21>
<TK_PLUS,14,22>
<TK_INT,2,14,23>
<TK_SEPARATOR,14,24>
<TK_CONST,15,5>
<TK_ID,L,15,11>
<TK_ASSIGN,15,13>
<TK_ID,M,15,16>
<TK_PLUS,15,17>
<TK_INT,3,15,18>
<TK_SEPARATOR,15,19>
<TK_IMPORT,16,5>
<TK_ID,a,16,12>
<TK_SEPARATOR,16,13>
<TK_CONST,17,5>
<TK_ID,K,17,11>
<TK_ASSIGN,17,13>
<TK_INT,0,17,16>
<TK_SEPARATOR,17,17>
<TK_END,18,1>
<TK_RESOURCE,20,1>
<TK_ID,c,20,10>
<TK_EXTEND,21,5>
<TK_ID,b,21,12>
<TK_SEPARATOR,21,13>
<TK_BODY,22,1>
<TK_LPA